In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#%matplotlib inline
sns.set()
from sklearn.feature_selection import mutual_info_regression
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.preprocessing import LabelEncoder
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/data-storm-4/Data-dictionary.xlsx
/kaggle/input/data-storm-4/DataStorm4.0_KaggleProblemStatement.pdf
/kaggle/input/data-storm-4/Store-info.csv
/kaggle/input/data-storm-4/Testing-data.csv
/kaggle/input/data-storm-4/Historical-transaction-data.csv


In [2]:
df = pd.read_csv('/kaggle/input/data-storm-4/Historical-transaction-data.csv')
store_info_data = pd.read_csv('/kaggle/input/data-storm-4/Store-info.csv')
test_data = pd.read_csv('/kaggle/input/data-storm-4/Testing-data.csv')

In [3]:
test_data.count()

shop_id         24
shop_profile     0
dtype: int64

In [4]:
store_info_data.isna().sum()

shop_id             0
shop_area_sq_ft     0
shop_profile       24
dtype: int64

In [5]:
# df = pd.merge(transaction_data, store_info_data, on='shop_id')

In [6]:
# df_test = dataframe[dataframe['shop_profile'].isnull()]

In [7]:
# df_train = dataframe[dataframe['shop_profile'].notnull()]

In [8]:
# df = df_train

In [9]:
# df =  df.reset_index(drop=True)

In [10]:
from datetime import datetime

def extract_date(date_str):
    date_obj = datetime.strptime(date_str, "%Y-%m-%dT%H:%M:%S.%fZ")
    return date_obj.strftime("%Y-%m-%d")

df["transaction_date"] = df["transaction_date"].apply(extract_date)

In [11]:
# Convert the date column to a Pandas datetime object
df["transaction_date"] = pd.to_datetime(df["transaction_date"])

# Extract the year, month, day, and week of the year into separate columns
df["month"] = df["transaction_date"].dt.month
df["day"] = df["transaction_date"].dt.day
df["week"] = df["transaction_date"].dt.isocalendar().week

In [12]:
# for colname in df.select_dtypes(["category", "object"]):
#         df[colname], _ = df[colname].factorize()

In [13]:
# Group the data by shop and customer, and count the number of transactions
transactions = df.groupby(["shop_id", "customer_id"])["invoice_id"].nunique()

# Count the number of customers who made more than one transaction at each shop
repeat_customers = transactions[transactions > 1].groupby("shop_id").count()

# Count the total number of unique customers who made a transaction at each shop
total_customers = transactions.groupby("shop_id").count()

# Calculate the customer loyalty rate for each shop
loyalty_rate = repeat_customers / total_customers

# Reset the index of the loyalty_rate Series and rename the columns
loyalty_rate = loyalty_rate.reset_index()
loyalty_rate.columns = ["shop_id", "loyalty_rate"]

# Merge the loyalty rate with the original DataFrame on the "shop_id" column
df = pd.merge(df, loyalty_rate, on="shop_id")

In [14]:
df = df.drop(["transaction_date","invoice_id"], axis= 1)

In [15]:
df.week.unique()

<IntegerArray>
[49, 50, 41, 42, 43, 44, 45, 46, 47, 48]
Length: 10, dtype: UInt32

In [16]:
df.shop_id.unique()

array(['SHOP008', 'SHOP112', 'SHOP055', 'SHOP003', 'SHOP071', 'SHOP123',
       'SHOP087', 'SHOP090', 'SHOP022', 'SHOP047', 'SHOP012', 'SHOP054',
       'SHOP040', 'SHOP023', 'SHOP115', 'SHOP104', 'SHOP097', 'SHOP103',
       'SHOP125', 'SHOP034', 'SHOP061', 'SHOP001', 'SHOP080', 'SHOP116',
       'SHOP075', 'SHOP093', 'SHOP096', 'SHOP026', 'SHOP032', 'SHOP107',
       'SHOP044', 'SHOP017', 'SHOP037', 'SHOP099', 'SHOP051', 'SHOP122',
       'SHOP088', 'SHOP121', 'SHOP046', 'SHOP067', 'SHOP098', 'SHOP065',
       'SHOP094', 'SHOP095', 'SHOP091', 'SHOP041', 'SHOP049', 'SHOP042',
       'SHOP004', 'SHOP076', 'SHOP073', 'SHOP027', 'SHOP083', 'SHOP043',
       'SHOP089', 'SHOP025', 'SHOP024', 'SHOP005', 'SHOP078', 'SHOP063',
       'SHOP021', 'SHOP086', 'SHOP105', 'SHOP074', 'SHOP062', 'SHOP079',
       'SHOP068', 'SHOP011', 'SHOP066', 'SHOP028', 'SHOP108', 'SHOP085',
       'SHOP038', 'SHOP060', 'SHOP111', 'SHOP059', 'SHOP118', 'SHOP013',
       'SHOP014', 'SHOP052', 'SHOP007', 'SHOP119', 

In [17]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

df['total_sales'] = df['item_price'] * df['quantity_sold']

# scaler = StandardScaler()
# scaled_data = scaler.fit_transform(df[['shop_area_sq_ft', 'item_price', 'quantity_sold', 'total_sales']])

# df = df.drop(['shop_area_sq_ft', 'item_price', 'quantity_sold', 'total_sales'], axis=1)

# scaled_df = pd.DataFrame(scaled_data, columns=['shop_area_sq_ft', 'item_price', 'quantity_sold', 'total_sales'])

# df = pd.concat([df, scaled_df], axis=1)

# print(df.head())

In [18]:
df.head()

,item_description,customer_id,shop_id,item_price,quantity_sold,month,day,week,loyalty_rate,total_sales
0,ORANGE BARLEY 1.5L,BGXA,SHOP008,220,2,12,11,49,0.058201,440
1,TONIC PET 500ML,VN7V,SHOP008,160,2,12,13,50,0.058201,320
2,CREAM SODA 1L,VN7V,SHOP008,150,2,12,13,50,0.058201,300
3,GINGER BEER 1.5L,VT9C,SHOP008,220,1,12,10,49,0.058201,220
4,GINGER BEER 1.5L,8QLS,SHOP008,440,1,12,10,49,0.058201,440


In [19]:
# Group transactions by shop_id
shop_transactions = df.groupby('shop_id')

# Calculate summary statistics for each shop
shop_summary = shop_transactions.agg({
    'item_description': 'nunique', # number of unique items sold
    'quantity_sold': 'sum', # total quantity sold
    'total_sales': 'sum', # total sales
})

# Calculate loyalty rate mean for each shop
loyalty_rate_mean = df.groupby('shop_id')['loyalty_rate'].mean()

# Merge loyalty rate mean into shop_summary
shop_summary = shop_summary.merge(loyalty_rate_mean, on='shop_id')

# Group the data by shop and calculate the average item price
avg_item_price = df.groupby("shop_id")["item_price"].mean()

# Reset the index of the avg_item_price Series and rename the columns
avg_item_price = avg_item_price.reset_index()
avg_item_price.columns = ["shop_id", "avg_item_price"]

# Merge the avg_item_price with the original DataFrame on the "shop_id" column
shop_summary = shop_summary.merge(avg_item_price, on='shop_id')

# Flatten column names
shop_summary.columns = [''.join(col).strip() for col in shop_summary.columns.values]

# Reset index to make shop_id a column again
shop_summary = shop_summary.reset_index()


In [20]:
shop_summary

,index,shop_id,item_description,quantity_sold,total_sales,loyalty_rate,avg_item_price
0,0,SHOP001,35,3942,1485285,0.228086,214.428218
1,1,SHOP002,35,9568,3084455,0.233605,227.111458
2,2,SHOP003,35,6959,1922995,0.215116,210.353922
3,3,SHOP004,35,6401,1571700,0.267202,205.565192
4,4,SHOP005,36,7784,2148520,0.256922,200.665318
...,...,...,...,...,...,...,...
119,119,SHOP123,36,3852,989775,0.231234,199.150169
120,120,SHOP124,35,9092,2967190,0.242438,212.094200
121,121,SHOP125,36,4402,1348395,0.200448,212.135584
122,122,SHOP126,31,6531,1893605,0.215941,203.701562


In [21]:
shop_summary = pd.merge(shop_summary, store_info_data, on='shop_id')

In [22]:
shop_summary

,index,shop_id,item_description,quantity_sold,total_sales,loyalty_rate,avg_item_price,shop_area_sq_ft,shop_profile
0,0,SHOP001,35,3942,1485285,0.228086,214.428218,580,Low
1,1,SHOP002,35,9568,3084455,0.233605,227.111458,637,NaN
2,2,SHOP003,35,6959,1922995,0.215116,210.353922,810,High
3,3,SHOP004,35,6401,1571700,0.267202,205.565192,516,Low
4,4,SHOP005,36,7784,2148520,0.256922,200.665318,617,Low
...,...,...,...,...,...,...,...,...,...
119,119,SHOP123,36,3852,989775,0.231234,199.150169,735,Moderate
120,120,SHOP124,35,9092,2967190,0.242438,212.094200,606,High
121,121,SHOP125,36,4402,1348395,0.200448,212.135584,1028,High
122,122,SHOP126,31,6531,1893605,0.215941,203.701562,328,Moderate


In [23]:
shop_summary

,index,shop_id,item_description,quantity_sold,total_sales,loyalty_rate,avg_item_price,shop_area_sq_ft,shop_profile
0,0,SHOP001,35,3942,1485285,0.228086,214.428218,580,Low
1,1,SHOP002,35,9568,3084455,0.233605,227.111458,637,NaN
2,2,SHOP003,35,6959,1922995,0.215116,210.353922,810,High
3,3,SHOP004,35,6401,1571700,0.267202,205.565192,516,Low
4,4,SHOP005,36,7784,2148520,0.256922,200.665318,617,Low
...,...,...,...,...,...,...,...,...,...
119,119,SHOP123,36,3852,989775,0.231234,199.150169,735,Moderate
120,120,SHOP124,35,9092,2967190,0.242438,212.094200,606,High
121,121,SHOP125,36,4402,1348395,0.200448,212.135584,1028,High
122,122,SHOP126,31,6531,1893605,0.215941,203.701562,328,Moderate


In [24]:
shop_summary['sales_per_feet'] = shop_summary['total_sales'] / shop_summary['shop_area_sq_ft']

In [25]:
shop_summary

,index,shop_id,item_description,quantity_sold,total_sales,loyalty_rate,avg_item_price,shop_area_sq_ft,shop_profile,sales_per_feet
0,0,SHOP001,35,3942,1485285,0.228086,214.428218,580,Low,2560.836207
1,1,SHOP002,35,9568,3084455,0.233605,227.111458,637,NaN,4842.158556
2,2,SHOP003,35,6959,1922995,0.215116,210.353922,810,High,2374.067901
3,3,SHOP004,35,6401,1571700,0.267202,205.565192,516,Low,3045.930233
4,4,SHOP005,36,7784,2148520,0.256922,200.665318,617,Low,3482.204214
...,...,...,...,...,...,...,...,...,...,...
119,119,SHOP123,36,3852,989775,0.231234,199.150169,735,Moderate,1346.632653
120,120,SHOP124,35,9092,2967190,0.242438,212.094200,606,High,4896.353135
121,121,SHOP125,36,4402,1348395,0.200448,212.135584,1028,High,1311.668288
122,122,SHOP126,31,6531,1893605,0.215941,203.701562,328,Moderate,5773.185976


In [26]:
shop_summary.item_description.unique()

array([35, 36, 31, 32, 33])

In [27]:
scaler = StandardScaler()
scaled_data = scaler.fit_transform(shop_summary[['quantity_sold', 'total_sales','shop_area_sq_ft', "avg_item_price",'sales_per_feet','loyalty_rate']])

shop_summary = shop_summary.drop(['quantity_sold', 'total_sales','shop_area_sq_ft','sales_per_feet', "avg_item_price",'loyalty_rate'], axis=1)

scaled_df = pd.DataFrame(scaled_data, columns=['quantity_sold', 'total_sales', 'shop_area_sq_ft', "avg_item_price",'sales_per_feet','loyalty_rate'])

shop_summary = pd.concat([shop_summary, scaled_df], axis=1)

In [28]:
df_test = shop_summary[shop_summary['shop_profile'].isnull()]
df_test =  df_test.reset_index(drop=True)

In [29]:
df_train = shop_summary[shop_summary['shop_profile'].notnull()]
df_train =  df_train.reset_index(drop=True)

In [30]:
df_x = df_train.drop('shop_profile', axis =1)
df_y = df_train['shop_profile']

In [31]:
df_x = df_x[['shop_id', 'sales_per_feet', "avg_item_price",'loyalty_rate']]

In [32]:
df1_x = df_test.drop('shop_profile', axis =1)
df1_x = df1_x[['shop_id','sales_per_feet', "avg_item_price",'loyalty_rate']]

In [33]:
X_train = df_x.drop('shop_id', axis =1 )
X_test = df1_x.drop('shop_id', axis =1 )
# X_train = df_x
# X_test = df1_x

In [34]:
# np.random.seed(123) 

# X_train, X_test, y_train, y_test = train_test_split(df_x, df_y,
#                                                    test_size= 0.2,
#                                                    shuffle= True, #shuffle the data to avoid bias
#                                                    random_state= 5)

# print(f'training set size: {X_train.shape[0]} samples \ntest set size: {X_test.shape[0]} samples')

y_train = df_y

In [35]:
X_test

,sales_per_feet,avg_item_price,loyalty_rate
0,0.773431,1.283619,-0.068856
1,1.097912,1.791746,1.427703
2,0.344960,0.706167,-0.357302
3,-1.068113,-0.747022,0.204964
4,-0.199885,0.047733,-0.470205
5,1.128119,2.093147,2.039288
6,-0.068561,-0.251637,0.473480
7,-1.133576,-0.558368,-1.111133
8,0.737257,0.782262,1.052182
9,1.404033,1.819606,3.075457


In [36]:
randomForest = RandomForestClassifier(max_depth=16, min_samples_split=2, min_samples_leaf=1)
# n_estimators = 100, max_depth=16, min_samples_split=2, min_samples_leaf=1

randomForest.fit(X_train, y_train)
y_pred = randomForest.predict(X_test)

In [37]:
# import xgboost as xgb
# from sklearn.datasets import load_breast_cancer
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# model = xgb.XGBClassifier()

# model.fit(X_train, y_train)

# y_pred = model.predict(X_test)

In [38]:
shop_profile = pd.DataFrame({'shop_profile': y_pred})

df_concat = pd.concat([df1_x, shop_profile], axis=1)


In [39]:
output = df_concat[['shop_id','shop_profile']]

In [40]:
output

,shop_id,shop_profile
0,SHOP002,Moderate
1,SHOP019,Moderate
2,SHOP023,Moderate
3,SHOP024,Moderate
4,SHOP029,High
5,SHOP030,Low
6,SHOP038,Low
7,SHOP044,Moderate
8,SHOP046,Moderate
9,SHOP050,Low


In [41]:
# Save test predictions to file

output.to_csv('submission.csv', index=False)

In [42]:
# accuracy = accuracy_score(y_test, y_pred)
# precision = precision_score(y_test, y_pred, average='weighted')
# recall = recall_score(y_test, y_pred, average='weighted')
# f1score = f1_score(y_test, y_pred, average='weighted')

# print("Accuracy:", accuracy)
# print("Precision:", precision)
# print("Recall:", recall)
# print("F1-score:", f1score)

In [43]:
# from sklearn.datasets import load_iris
# from sklearn.model_selection import cross_val_score
# from sklearn.neighbors import KNeighborsClassifier

# knn = KNeighborsClassifier(n_neighbors=5)

# # Fit the model to the training data
# knn.fit(X_train, y_train)

# # Use the fitted model to make predictions on the test data
# y_pred = knn.predict(X_test)

In [44]:
# accuracy = accuracy_score(y_test, y_pred)
# precision = precision_score(y_test, y_pred, average='weighted')
# recall = recall_score(y_test, y_pred, average='weighted')
# f1score = f1_score(y_test, y_pred, average='weighted')

# print("Accuracy:", accuracy)
# print("Precision:", precision)
# print("Recall:", recall)
# print("F1-score:", f1score)

In [45]:
# max_depths = [2*i for i in range(1,15)]
# min_samples_splits = [2,4,8]
# min_samples_leafs =  [1,2,4]
# n_estimator = 100
# best_score = 0

# for max_depth in max_depths:
#     for min_samples_split in min_samples_splits:
#         for min_samples_leaf in min_samples_leafs:
#             random_forest = RandomForestClassifier( max_depth=max_depth, min_samples_split=min_samples_split, min_samples_leaf=min_samples_leaf)

#             accuracy_list = cross_val_score(random_forest, df_x, df_y, cv=5)
#             accuracy = accuracy_list.mean()

#             if accuracy > best_score:
#                 best_score = accuracy
#                 best_params = {'max_depth': max_depth, 'min_samples_split': min_samples_split, 'min_samples_leaf': min_samples_leaf}
#                 print(f'Best hyperparameters: {best_params}')
#                 print(f'Accuracy: {best_score}')


# print(f'Best hyperparameters: {best_params}')
# print(f'Accuracy: {best_score}')